In [3]:
import mlflow, pprint

mlflow.set_tracking_uri("http://10.90.90.95:5000")

# ↓ zobacz, gdzie wskazuje Default
default_exp = mlflow.get_experiment("0")
pprint.pprint(default_exp.__dict__, width=120)


{'_artifact_location': '/mlflow/artifacts/0',
 '_creation_time': 1752669313697,
 '_experiment_id': '0',
 '_last_update_time': 1752669313697,
 '_lifecycle_stage': 'active',
 '_name': 'Default',
 '_tags': {}}


In [7]:
import mlflow, os, pathlib, tempfile
mlflow.set_tracking_uri("http://10.90.90.95:5000")

exp_name = "test_http_artifacts"
if mlflow.get_experiment_by_name(exp_name) is None:
    mlflow.create_experiment(
        name=exp_name,
        artifact_location=f"mlflow-artifacts:/{exp_name}"
    )

mlflow.set_experiment(exp_name)

with mlflow.start_run() as run:
    tmp = pathlib.Path(tempfile.mktemp(suffix=".txt"))
    tmp.write_text("hello")
    mlflow.log_artifact(str(tmp))
    print("ARTIFACT_URI ->", mlflow.get_run(run.info.run_id).info.artifact_uri)


MlflowException: Cannot set a deleted experiment 'test_http_artifacts' as the active experiment. You can restore the experiment, or permanently delete the experiment to create a new one.

In [11]:
from mlflow.tracking import MlflowClient
mlflow.set_tracking_uri("http://10.90.90.95:5000")

client = MlflowClient()

exp = client.get_experiment_by_name("traisformer_baltic_small")
if exp:
    client.delete_experiment(exp.experiment_id)

RestException: RESOURCE_DOES_NOT_EXIST: No Experiment with id=3 exists

In [12]:
import mlflow
from mlflow.exceptions import MlflowException
from mlflow.tracking import MlflowClient

exp_name = "traisformer_baltic_small"
client = MlflowClient()

try:
    mlflow.set_experiment(exp_name)
except MlflowException:
    exp = client.get_experiment_by_name(exp_name)
    if exp and exp.lifecycle_stage == "deleted":
        # restore the soft-deleted experiment
        client.restore_experiment(exp.experiment_id)
    else:
        # it wasn't deleted but doesn't exist, so create anew
        client.create_experiment(exp_name)
    mlflow.set_experiment(exp_name)